# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [59]:
# import libraries
import nltk
nltk.download(['wordnet', 'punkt'])

import pandas as pd
import numpy as np
from sqlalchemy import create_engine

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/arunasumbrasas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/arunasumbrasas/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [60]:
# load data from database
engine = create_engine('sqlite:///messages.db')
df = pd.read_sql_table('messages_clean', con=engine)
X = df.message
y = df[['related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report']]

### 2. Write a tokenization function to process your text data

In [65]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for token in tokens:
        clean_token = lemmatizer.lemmatize(token).lower().strip()
        clean_tokens.append(clean_token)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [34]:
pipeline = Pipeline([
    ('vect', TfidfVectorizer(stop_words='english')),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])

In [66]:
pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ]))
    ])),

    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
model = pipeline
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [68]:
category_names = y.columns

In [69]:
# MOC/KN - no feature union - standard params: 0.2331
# MOC/KN - feature union - standard params: 0.2166

model.score(X_test, y_test)

0.21666158071406774

In [70]:
for i in range(len(category_names)):
    acc = classification_report(y_pred[:,i], y_test.iloc[:,i])
    print("accuracy for {} : {}".format(category_names[i], acc))

accuracy for related :               precision    recall  f1-score   support

           0       0.30      0.63      0.41       710
           1       0.95      0.81      0.88      5836
           2       0.09      0.62      0.16         8

    accuracy                           0.79      6554
   macro avg       0.45      0.69      0.48      6554
weighted avg       0.88      0.79      0.82      6554

accuracy for request :               precision    recall  f1-score   support

           0       0.95      0.91      0.93      5710
           1       0.53      0.70      0.60       844

    accuracy                           0.88      6554
   macro avg       0.74      0.80      0.76      6554
weighted avg       0.90      0.88      0.89      6554

accuracy for offer :               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6552
           1       0.03      0.50      0.06         2

    accuracy                           0.99      6554
   macr

/Users/arunasumbrasas/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [79]:
model.get_params()

{'cv': 'warn',
 'error_score': 'raise-deprecating',
 'estimator__memory': None,
 'estimator__steps': [('features', FeatureUnion(n_jobs=None,
                transformer_list=[('text_pipeline',
                                   Pipeline(memory=None,
                                            steps=[('vect',
                                                    CountVectorizer(analyzer='word',
                                                                    binary=False,
                                                                    decode_error='strict',
                                                                    dtype=<class 'numpy.int64'>,
                                                                    encoding='utf-8',
                                                                    input='content',
                                                                    lowercase=True,
                                                                    max_df=1.0,


### 6. Improve your model
Use grid search to find better parameters. 

In [74]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB

In [80]:
parameters = [
    {"clf": [MultiOutputClassifier(RandomForestClassifier())],
     "clf__estimator__n_estimators": [10, 100, 250],
     "clf__estimator__max_depth":[8],
     "clf__estimator__random_state":[42]},
    {"clf": [MultiOutputClassifier(KNeighborsClassifier())],
     "clf__estimator__leaf_size": [5, 10, 30],
     "clf__estimator__n_neighbors": [3, 5, 7]}
]

cv = GridSearchCV(pipeline, param_grid=parameters)

In [ ]:
model = cv
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

/Users/arunasumbrasas/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
model.score(X_test, y_test)

In [ ]:
accuracy = (y_pred == y_test).mean()
print("Accuracy:", accuracy)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
import pickle

filename = 'disaster_model.sav'
pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.